# Nifty File Format

Nifty (Neuroimaging Informatics Technology Initiative) is a file format to handle neuro images.
One nifty object contains:
- N-D array containing the image data (think of it as numpy array)
- (4,4) affine matrix mapping array coordinates to coordinates in some world space
- metadata in form of a header

Common package to use niftys is nibabel

The BIDS Toolbox also has a NII_Wrapper, that should be used instead of nibabel directly.

In [1]:
# Lets build a simple nifty ourselves
import numpy as np
import nibabel as nib

import BIDS
from BIDS import *

test_data_arr = np.random.random(size=(30, 20, 10))  # for test: generate arbitrary image data

nibabel_nifty = nib.Nifti1Image(dataobj=test_data_arr, affine=np.eye(4))  # creating a nifty

bids_nii = BIDS.NII(nibabel_nifty)  # now we have a NII in the BIDS toolbox

## Read, Write Niftys

Normally, you would load the NII directly from a BIDS_File by using bids_file.open_nii()

You can use NII as a smart data container
- get_array() gives you the nifty image data as numpy array
- shape() gives you the dimensions of the image data
- header(), affine() returns the nifty header, affine matrix
- zoom() returns the resolution of the dimensions (so zoom of (1,1,3) means that in the first dimensions, one pixel stands for 1mm in real life, 3 means one pixel stands for 3mm. Lower values -> better resolution)
- there are also setter functions for the array, ...

You can save nifty as a .nii.gz on the disk by calling:
- save(file: str | Path)

In [2]:
print("shape", bids_nii.shape)
print(
    "zoom", bids_nii.zoom
)  # of course this is (1,1,1) because as we constructed it from scratch, it doesnt know its zoom and uses isometric space as default. If you load niftys, these number differ

shape (30, 20, 10)
zoom (1.0, 1.0, 1.0)


## Orientation

All NIIs have a orientation member variable.
Those are axcodes that represent the positive direction of the axes.
- L: Left
- R: Right
- S: Superior (up)
- I: Inferior (down)
- A: Anterior (front)
- P: Posterior (back)

In [3]:
print("orientation", bids_nii.orientation)
# This means that the first dimension of the image data array stands for the left/right axis with positive ax coordinates going to the Right, ...

orientation ('R', 'A', 'S')


## Manipulate Nifty

Sometimes, it is vital to resample nifty into isotropic (1,1,1) space.
This can be done with nifty.rescale()

Important functions are:
- rescale(voxel_spacing: tuple(float, float, float)) # Rescales the image date to the given resolution
- reorient(axcodes_to: Ax_Codes) # Reorients the data to the given ax codes (see orientation above)
- extract_label(label: int) # returns a new array that only contains the given label (useful for segmentations)

- compute_crop_slice() # is used to compute a bounding box around the used space of the image data
- apply_crop_slice() # is used after compute_crop_slice() to perform the actual cutout of the bounding box

Functions could exist both with and without a "_" at the end. The "_" version is the inplace variant (will directly modify that object).
Example:

nii.rescale(("P", "I", "R")) returns a rescaled copy of the array, while

nii.rescale_(("P", "I", "R")) modifies the nii object directly, rescaling it.

In [4]:
bids_nii.rescale_((2, 1, 1), verbose=True)
print("zoom", bids_nii.zoom)
# Of course if we change the zoom (resolution), the shape changes with it
print("shape", bids_nii.shape)

bids_nii.reorient_(("P", "I", "R"), verbose=True)  # nearly all functions have a verbose parameter...
print("orientation", bids_nii.orientation)
print("shape", bids_nii.shape)

[ ] Image resampled from (1.0, 1.0, 1.0) to voxel size (2, 1, 1)
zoom (2.0, 1.0, 1.0)
shape (15, 20, 10)
[ ] Image reoriented from ('R', 'A', 'S') to ('P', 'I', 'R')
orientation ('P', 'I', 'R')
shape (20, 10, 15)


# Cropping empty space

In [5]:
crop = bids_nii.compute_crop_slice(dist=10)
small_nii = bids_nii.apply_crop_slice(crop)
# do something with small_nii
# ...
# Save back to big array
arr = bids_nii.get_array()
arr[crop] = small_nii.get_array()
bids_nii.set_array(arr, inplace=True)